In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [3]:
from sklearn.base import BaseEstimator,TransformerMixin
class removename(BaseEstimator,TransformerMixin):
    def __init__(self):
        self.names = []
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        for name_id in range(len(X['Name'])):
            self.names.append(X['Name'][name_id].split(',')[0])
        X['Name'] = self.names
        return X

In [63]:
n = removename()
n.transform(data)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Braund,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,Cumings,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Heikkinen,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,Futrelle,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,Allen,male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,Moran,male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,McCarthy,male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,Palsson,male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,Johnson,female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,Nasser,female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder

class AttributesAdder(BaseEstimator,TransformerMixin):
    def __init__(self):
        self.family = []
    def fit(self,X,y=None):
        pass
    def transform(self,X,y=None):
        for idx in range(len(X['SibSp'])):
            self.family.append(X['SibSp'][idx] + X['Parch'][idx])
        X['Family'] = self.family
        del X['SibSp'] , X['Parch']
        return X
    
class Selector(BaseEstimator,TransformerMixin):
    def __init__(self,col_names):
        self.col_names = col_names
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        return X[self.col_names].values

In [5]:
numeric_col = ['Age','Fare']
numeric_transformer = Pipeline([
    ('selector',Selector(numeric_col)),
    ('imputer',SimpleImputer(strategy='median')),
    ('std_scaler',StandardScaler()),
])
categorical_col = ['Sex','Embarked','Pclass']
categorical_transformer = Pipeline([
    ('selector',Selector(categorical_col)),
    ('imputer',SimpleImputer(strategy='constant',fill_value='missing')),
    ('encoder',OneHotEncoder(handle_unknown='ignore')),
])

preprocessor = ColumnTransformer([
    ('num',numeric_transformer,numeric_col),
    ('cat',categorical_transformer,categorical_col),
])

clf = Pipeline([
    ('prepro',preprocessor),
])

In [6]:
aa = AttributesAdder()

In [7]:
aa.transform(data)

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Family
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,NaN,S,0
5,6,0,3,"Moran, Mr. James",male,NaN,330877,8.4583,NaN,Q,0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,17463,51.8625,E46,S,0
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,349909,21.0750,NaN,S,4
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,347742,11.1333,NaN,S,2
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,237736,30.0708,NaN,C,1


In [18]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Family
0,1,0,3,Braund,male,22.0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,Cumings,female,38.0,PC 17599,71.2833,C85,C,1
2,3,1,3,Heikkinen,female,26.0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,Futrelle,female,35.0,113803,53.1000,C123,S,1
4,5,0,3,Allen,male,35.0,373450,8.0500,NaN,S,0


In [8]:
y = data['Survived'].copy()

In [68]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [9]:
prepare = clf.fit_transform(data)

In [10]:
df = pd.DataFrame(prepare)

In [11]:
df.columns = [['Age','Fare','Female','Male','Embarked_C','Embarked_Q','Embared_S','Pclass_0','Pclass_1','Pclass_2','Pclass_3']]

In [12]:
df.drop('Pclass_0',inplace=True,axis=1)

C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\pandas\core\generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [13]:
df.index = data['PassengerId']

In [14]:
df['Family'] = data['Family']

In [136]:
def repl(X,col_name):
    for idx in range(len(X[col_name])):
        if pd.isna(X[col_name][idx]) == False:
            X[col_name][idx] = 1
        else:
            X[col_name][idx] = 0
    return X

In [245]:
df.tail()

,Age,Fare,Female,Male,Embarked_C,Embarked_Q,Embared_S,Pclass_1,Pclass_2,Pclass_3,Family
886,-0.181487,-0.386671,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0
887,-0.796286,-0.044381,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0
888,-0.104637,-0.176263,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3
889,-0.258337,-0.044381,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0
890,0.202762,-0.492378,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0


In [15]:
df['Cabin']=(data['Cabin'].isna() == True).astype(int)

In [17]:
df.fillna(0, inplace=True)

In [18]:
X = df.copy()

In [19]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()

In [20]:
log_reg.fit(X,y)

C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [21]:
prediction = log_reg.predict(X)

In [263]:
prediction

array([0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,

In [22]:
log_reg.score(X,y)

0.792368125701459

In [35]:
log_rmse

0.45566640681373577

In [23]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg,X,y,scoring="neg_mean_squared_error",cv=10)
log_rmse_scores = np.sqrt(-scores)

C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to '

In [24]:
log_rmse_scores

array([0.48304589, 0.45946829, 0.47404546, 0.39661489, 0.43704832,
       0.46204236, 0.48575205, 0.48575205, 0.44971901, 0.45226702])

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
forest_reg = RandomForestClassifier(n_estimators=10)
forest_reg.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [45]:
forest_reg.predict(X)

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [26]:
forest_reg.score(X,y)

0.9753086419753086

In [42]:
forest_scores = cross_val_score(forest_reg,X,y,scoring="neg_mean_squared_error",cv=10)
forest_rmse_score = np.sqrt(-forest_scores)

In [43]:
forest_rmse_score

array([0.48304589, 0.43461349, 0.49718308, 0.36719404, 0.33520076,
       0.44971901, 0.43704832, 0.48575205, 0.44971901, 0.41286141])

In [48]:
predicts = pd.DataFrame(log_reg.predict(X))

In [53]:
predicts.columns = ['log_reg']

In [54]:
predicts['forest_reg'] = forest_reg.predict(X)
predicts['Real'] = y

In [145]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'C':np.logspace(-20,20,5),
     'penalty':['l1','l2']}
]
grid_search = GridSearchCV(log_reg,param_grid,cv=10,n_jobs=1)
grid_search.fit(df,y)

C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to '

C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to '

C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to '

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid=[{'C': array([1.e-20, 1.e-10, 1.e+00, 1.e+10, 1.e+20]), 'penalty': ['l1', 'l2']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [146]:
grid_search.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [153]:
final_model = grid_search.best_estimator_
grid_search.best_estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [71]:
bb = AttributesAdder()

In [72]:
bb.transform(test)

,PassengerId,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Family
0,892,3,"Kelly, Mr. James",male,34.5,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,3101298,12.2875,NaN,S,2
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,7538,9.2250,NaN,S,0
6,898,3,"Connolly, Miss. Kate",female,30.0,330972,7.6292,NaN,Q,0
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,248738,29.0000,NaN,S,2
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,2657,7.2292,NaN,C,0
9,901,3,"Davies, Mr. John Samuel",male,21.0,A/4 48871,24.1500,NaN,S,2


In [73]:
prepare_test = clf.fit_transform(test)
prepare_test

array([[ 0.38623105, -0.49741333,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 1.37137004, -0.51227801,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 2.55353683, -0.46410047,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.70147553, -0.50779638,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [-0.20485235, -0.49345515,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [-0.20485235, -0.23695704,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [74]:
df_test = pd.DataFrame(prepare_test)
df_test

,0,1,2,3,4,5,6,7,8,9
0,0.386231,-0.497413,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.371370,-0.512278,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,2.553537,-0.464100,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,-0.204852,-0.482475,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4,-0.598908,-0.417492,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5,-1.229397,-0.472391,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
6,0.031581,-0.500999,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
7,-0.283663,-0.117894,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
8,-0.914152,-0.508169,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
9,-0.677719,-0.204838,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [75]:
df_test.columns = [['Age','Fare','Female','Male','Embarked_C','Embarked_Q','Embared_S','Pclass_1','Pclass_2','Pclass_3']]

In [76]:
df_test['Family'] = test['Family']

In [77]:
df_test['Cabin'] = test['Cabin']

In [78]:
df_test['Cabin']=(df_test['Cabin'].isna()==True).astype(int)

In [79]:
df_test

,Age,Fare,Female,Male,Embarked_C,Embarked_Q,Embared_S,Pclass_1,Pclass_2,Pclass_3,Family,Cabin
0,0.386231,-0.497413,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0,1
1,1.371370,-0.512278,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1,1
2,2.553537,-0.464100,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0,1
3,-0.204852,-0.482475,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0,1
4,-0.598908,-0.417492,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2,1
5,-1.229397,-0.472391,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0,1
6,0.031581,-0.500999,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0,1
7,-0.283663,-0.117894,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2,1
8,-0.914152,-0.508169,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0,1
9,-0.677719,-0.204838,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2,1


In [80]:
log_reg.predict(df_test)
log_reg

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [123]:
Goal = pd.read_csv('./gender_submission.csv')

In [124]:
Real = Goal['Survived'].copy()
Real

0      0
1      1
2      0
3      0
4      1
5      0
6      1
7      0
8      1
9      0
10     0
11     0
12     1
13     0
14     1
15     1
16     0
17     0
18     1
19     1
20     0
21     0
22     1
23     0
24     1
25     0
26     1
27     0
28     0
29     0
      ..
388    0
389    0
390    0
391    1
392    0
393    0
394    0
395    1
396    0
397    1
398    0
399    0
400    1
401    0
402    1
403    0
404    0
405    0
406    0
407    0
408    1
409    1
410    1
411    1
412    1
413    0
414    1
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64

In [97]:
log_reg.score(df_test,Real)

0.9138755980861244

In [82]:
Goal['log_reg'] = log_reg.predict(df_test)

In [122]:
Goal

,PassengerId,Survived,for_reg
0,892,0,0
1,893,0,0
2,894,0,0
3,895,0,0
4,896,1,1
5,897,0,0
6,898,1,0
7,899,0,0
8,900,1,1
9,901,0,0


In [112]:
Goal['Survived'] = final_model.predict(df_test)

In [96]:
forest_reg.score(df_test,Real)

0.8660287081339713

In [113]:
Submission = Goal.drop('Survived',axis=1)
Submission.columns = ['PassengerId','Survived']

In [114]:
Submission.to_csv('./Submission_2.csv',index=False)

In [96]:
Submission.drop('index',axis=1)

KeyError: "['index'] not found in axis"

In [92]:
Submission.to_csv('./Submission.csv')

In [49]:
grid_search.best_estimator_.set_params

<bound method BaseEstimator.set_params of LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)>

In [98]:
df_2 = df.drop(['Embarked_Q','Pclass_2'],axis=1)
df_2.fillna(0,inplace=True)

C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\pandas\core\generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [105]:
y = data['Survived'].copy()

In [112]:
log_reg.fit(df_2,y)

C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [113]:
log_reg.predict(df_2)

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,

In [114]:
log_reg.score(df_2,y)

0.7901234567901234

In [115]:
y_test = Goal['Survived'].copy()

In [116]:
log_reg.score(df_test2,y_test)

0.9138755980861244

In [117]:
log_reg.predict(df_test2)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [119]:
from sklearn.tree import DecisionTreeClassifier
tree_cla = DecisionTreeClassifier()
tree_cla.fit(df_2,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [120]:
tree_cla.score(df_2,y)

0.9887766554433222

In [122]:
tree_cla.score(df_test2,y_test)

0.7751196172248804

In [69]:
final_model.fit(df_2,y)
final_model.score(df_2,y)

C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7957351290684624

In [68]:
final_model.score(X,y)
grid_search.best_estimator_.coef_

ValueError: X has 12 features per sample; expecting 10

In [70]:
final_model.score(df_test2,y_test)

NameError: name 'df_test2' is not defined

In [141]:
final_model.predict(df_test2)

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [147]:
Submit = test['PassengerId']
Submit['Survived'] = final_model.predict(df_test2)

C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [109]:
df_test.drop(['Embarked_Q','Pclass_2'],axis=1,inplace=True)

C:\Users\Seung kyu Hong\Anaconda3\lib\site-packages\pandas\core\generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [156]:
final_model.predict(df_test)

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [144]:
final_model.coef_

array([[-0.42188329,  0.02009303,  2.12082484, -0.41379659,  0.        ,
        -0.49033026,  0.92491768, -1.30357179, -0.04711094, -0.30693803]])

In [157]:
Submission['Survived'] = final_model.predict(df_test)
Submission.to_csv('./Submission_2.csv',index=False)

In [155]:
final_model.score(df_test,Real)

1.0